In [4]:
import boto3
import json
import numpy as np
import faiss

# Initialize Bedrock client
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

def get_embedding(text):
    body = json.dumps({"inputText": text})
    response = bedrock.invoke_model(
        modelId='amazon.titan-embed-text-v2:0',
        contentType='application/json',
        accept='application/json',
        body=body
    )
    return json.loads(response['body'].read())['embedding']

# Sentences
sentences = [
    "The cat is on the mat.",
    "A kitten is resting on the rug.",
    "The sun is shining brightly."
]

# Get embeddings
embeddings = np.array([get_embedding(sentence) for sentence in sentences], dtype='float32')

# Initialize FAISS index (L2 distance)
dimension = len(embeddings[0])  # e.g., 1536 for Titan Embeddings
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Query for similarity
query_emb = np.array([get_embedding("The cat is on the mat.")], dtype='float32')
distances, indices = index.search(query_emb, k=3)
for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
    print(f"Sentence: {sentences[idx]}, Distance: {dist:.3f}")

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: You don't have access to the model with the specified model ID.